In [ ]:
import sys 
sys.path.append(rf"/nfs/home/genovese/thesis-wildfire-genovese/notebooks/")
from importlib import reload
import utils
reload(utils)
from utils import *
data_folder = "/nfs/home/genovese/thesis-wildfire-genovese/data/"

In [ ]:
from shapely import wkt

for a in list(range(2000, 2025)):
    df = pd.read_csv(f'/nfs/home/genovese/thesis-wildfire-genovese/data/clean_data/weather_forecast/{a}.csv')
    df['geometry'] = wkt.loads(df['geometry'])
    gdf = gpd.GeoDataFrame(df, geometry='geometry')
    save_clean_data(gdf,
                    str(a),
                    f'/nfs/home/genovese/thesis-wildfire-genovese/data/gathering_geojson/weather_forecast')

---

USELESS

In [ ]:
from sklearn.preprocessing import OneHotEncoder

gdf = gpd.read_file('/nfs/home/genovese/thesis-wildfire-genovese/data/gathering_geojson/vegetazione.geojson')

encoder = OneHotEncoder()  # define order
encoded = encoder.fit_transform(gdf[['CATEGORIA']]).toarray()
pd.DataFrame(encoded).describe()

In [ ]:
import rasterio
from rasterio.features import rasterize
from rasterio.transform import from_origin


def rasterise_shp(shapefile_path, output_raster, resolution = 100, attributes = None):

    # === Load shapefile ===
    gdf = gpd.read_file(shapefile_path)
    bounds = gdf.total_bounds  # (minx, miny, maxx, maxy)

    # === Define raster resolution and size ===
    minx, miny, maxx, maxy = bounds
    width = int((maxx - minx) / resolution)
    height = int((maxy - miny) / resolution)
    transform = from_origin(minx, maxy, resolution, resolution)

    # attributes = list(gdf.columns)
    # attributes.remove('geometry')
    # === Convert geometries and optional values ===
    for att in attributes:
        if att:
            shapes = ((geom, value) for geom, value in zip(gdf.geometry, gdf[att]))
        else:
            shapes = ((geom, 1) for geom in gdf.geometry)

        raster = rasterize(
            shapes=shapes,
            out_shape=(height, width),
            transform=transform,
            fill=-1000,
            # dtype='float32'
        )
    
        # === Save raster to file ===
        with rasterio.open(
            output_raster,
            'w',
            driver='GTiff',
            height=height,
            width=width,
            count=1,
            dtype=raster.dtype,
            crs=gdf.crs,
            transform=transform,
        ) as dst:
            dst.write(raster, 1)

    print(f"Raster saved to {output_raster}")


In [ ]:
rasterise_shp('/nfs/home/genovese/thesis-wildfire-genovese/data/gathering_geojson/vegetazione.geojson', 
              '/nfs/home/genovese/thesis-wildfire-genovese/outputs/rasterisations/vegetazione.tif',
              attributes=['ESTENSIONE'])

In [ ]:
# Open the TIFF file
with rasterio.open("/nfs/home/genovese/thesis-wildfire-genovese/outputs/rasterisations/vegetazione.tif") as src:
    num_channels = src.count
    print(f"Number of channels: {num_channels}")

    # Read and plot each channel
    fig, axs = plt.subplots(1, num_channels, figsize=(5 * num_channels, 5))
    if num_channels == 1:
        axs = [axs]  # Ensure it's iterable

    for i in range(1, num_channels + 1):
        band = src.read(i)
        axs[i-1].imshow(band, cmap='gray')
        axs[i-1].set_title(f'Band {i}')
        axs[i-1].axis('off')

    plt.tight_layout()
    plt.show()

---